In [5]:
import os
while not os.path.exists('.gitignore'):
	os.chdir(os.path.abspath(os.path.join('../')))
from common_imports import *

In [6]:
df = pd.read_parquet('_data/raw/juros-bancarios/combined.parquet')

In [7]:
df

,Segmento,Modalidade,InstituicaoFinanceira,TaxaJurosAoAno
0,Pessoa Física,FINANCIAMENTO IMOBILIÁRIO COM TAXAS DE MERCADO...,CPCI,"10,54"
1,Pessoa Física,FINANCIAMENTO IMOBILIÁRIO COM TAXAS DE MERCADO...,BCO CITIBANK S.A.,"11,66"
2,Pessoa Física,FINANCIAMENTO IMOBILIÁRIO COM TAXAS DE MERCADO...,BCO DO BRASIL S.A.,"16,40"
3,Pessoa Física,FINANCIAMENTO IMOBILIÁRIO COM TAXAS DE MERCADO...,BRAZILIAN MORTGAGES CH S.A.,"18,50"
4,Pessoa Física,FINANCIAMENTO IMOBILIÁRIO COM TAXAS DE MERCADO...,BANCO INTER,"31,13"
...,...,...,...,...
2411705,Pessoa Jurídica,Adiantamento sobre contratos de câmbio (ACC) -...,BANCO INTER,"10,40"
2411706,Pessoa Jurídica,Adiantamento sobre contratos de câmbio (ACC) -...,BCO CARGILL S.A.,"10,50"
2411707,Pessoa Jurídica,Adiantamento sobre contratos de câmbio (ACC) -...,BCO BS2 S.A.,"10,61"
2411708,Pessoa Jurídica,Adiantamento sobre contratos de câmbio (ACC) -...,TRAVELEX BANCO DE CÂMBIO S.A.,"13,00"


In [8]:
df['TaxaJurosAoAno'] = df['TaxaJurosAoAno'].str.replace(',', '.').astype(float)

In [9]:
df['InstituicaoFinanceira'].value_counts()

InstituicaoFinanceira
BCO BRADESCO S.A.                 68906
BCO DO BRASIL S.A.                68781
BCO SANTANDER (BRASIL) S.A.       66787
BCO DO ESTADO DO RS S.A.          63263
ITAÚ UNIBANCO S.A.                59306
                                  ...  
MERCANTIL BRASIL LEASING SA AM        8
TRADIÇÃO                              5
BMG LEASING S/A-AM                    5
BANIF INVESTIMENTO                    5
GOLDMAN SACHS DO BRASIL BM S.A        5
Name: count, Length: 270, dtype: int64

In [10]:
df.head()

,Segmento,Modalidade,InstituicaoFinanceira,TaxaJurosAoAno
0,Pessoa Física,FINANCIAMENTO IMOBILIÁRIO COM TAXAS DE MERCADO...,CPCI,10.54
1,Pessoa Física,FINANCIAMENTO IMOBILIÁRIO COM TAXAS DE MERCADO...,BCO CITIBANK S.A.,11.66
2,Pessoa Física,FINANCIAMENTO IMOBILIÁRIO COM TAXAS DE MERCADO...,BCO DO BRASIL S.A.,16.40
3,Pessoa Física,FINANCIAMENTO IMOBILIÁRIO COM TAXAS DE MERCADO...,BRAZILIAN MORTGAGES CH S.A.,18.50
4,Pessoa Física,FINANCIAMENTO IMOBILIÁRIO COM TAXAS DE MERCADO...,BANCO INTER,31.13


In [11]:
df['Modalidade'].value_counts()

Modalidade
Crédito pessoal não-consignado - Pré-fixado                                                    223087
Cheque especial - Pré-fixado                                                                   168685
Crédito pessoal consignado privado - Pré-fixado                                                155191
Desconto de duplicatas - Pré-fixado                                                            152863
Capital de giro com prazo até 365 dias - Pré-fixado                                            143349
Aquisição de veículos - Pré-fixado                                                             141057
Cartão de crédito - rotativo total - Pré-fixado                                                132411
Capital de giro com prazo superior a 365 dias - Pré-fixado                                     127021
Crédito pessoal consignado público - Pré-fixado                                                125878
Aquisição de outros bens - Pré-fixado                                  

In [12]:
df['SubClassificacao'] = df['Modalidade'].apply(lambda x: x.split(' - ')[-1])
df['Modalidade'] = df['Modalidade'].apply(lambda x: x.rsplit(' - ', 1)[0])

In [13]:
df['Modalidade'].value_counts()

Modalidade
Capital de giro com prazo até 365 dias            230590
Crédito pessoal não-consignado                    223087
Capital de giro com prazo superior a 365 dias     217137
Conta garantida                                   217096
Cheque especial                                   168685
Crédito pessoal consignado privado                155191
Desconto de duplicatas                            152863
Aquisição de veículos                             141057
Cartão de crédito - rotativo total                132411
Crédito pessoal consignado público                125878
Aquisição de outros bens                          124869
Cartão de crédito - parcelado                     120252
Crédito pessoal consignado INSS                   115134
Adiantamento sobre contratos de câmbio (ACC)      106677
Desconto de cheques                                95385
Vendor                                             40298
Antecipação de faturas de cartão de crédito        28354
Arrendamento mercant

In [14]:
df['SubClassificacao'].value_counts()

SubClassificacao
Pré-fixado                                      2015560
Pós-fixado referenciado em juros flutuantes      284465
Pós-fixado referenciado em moeda estrangeira     106677
PÓS-FIXADO REFERENCIADO EM TR                      3544
PRÉ-FIXADO                                          936
PÓS-FIXADO REFERENCIADO EM IPCA                     528
Name: count, dtype: int64

In [15]:
def transform_taxa(valor_taxa, periodo_desejado, periodo_atual):
	vt = Decimal(valor_taxa)
	pd = Decimal(periodo_desejado)
	pa = Decimal(periodo_atual)
 
	a = 1 + (vt / 100)
	b = pd / pa
	c = a ** b
	d = (c - 1) * 100

	return d

In [16]:
def trim_decimal(n):
	valor = Decimal(str(n).rstrip('0').rstrip('.'))
	return valor.quantize(Decimal('1.0000'), rounding=ROUND_HALF_UP)

In [17]:
df

,Segmento,Modalidade,InstituicaoFinanceira,TaxaJurosAoAno,SubClassificacao
0,Pessoa Física,FINANCIAMENTO IMOBILIÁRIO COM TAXAS DE MERCADO,CPCI,10.54,PRÉ-FIXADO
1,Pessoa Física,FINANCIAMENTO IMOBILIÁRIO COM TAXAS DE MERCADO,BCO CITIBANK S.A.,11.66,PRÉ-FIXADO
2,Pessoa Física,FINANCIAMENTO IMOBILIÁRIO COM TAXAS DE MERCADO,BCO DO BRASIL S.A.,16.40,PRÉ-FIXADO
3,Pessoa Física,FINANCIAMENTO IMOBILIÁRIO COM TAXAS DE MERCADO,BRAZILIAN MORTGAGES CH S.A.,18.50,PRÉ-FIXADO
4,Pessoa Física,FINANCIAMENTO IMOBILIÁRIO COM TAXAS DE MERCADO,BANCO INTER,31.13,PRÉ-FIXADO
...,...,...,...,...,...
2411705,Pessoa Jurídica,Adiantamento sobre contratos de câmbio (ACC),BANCO INTER,10.40,Pós-fixado referenciado em moeda estrangeira
2411706,Pessoa Jurídica,Adiantamento sobre contratos de câmbio (ACC),BCO CARGILL S.A.,10.50,Pós-fixado referenciado em moeda estrangeira
2411707,Pessoa Jurídica,Adiantamento sobre contratos de câmbio (ACC),BCO BS2 S.A.,10.61,Pós-fixado referenciado em moeda estrangeira
2411708,Pessoa Jurídica,Adiantamento sobre contratos de câmbio (ACC),TRAVELEX BANCO DE CÂMBIO S.A.,13.00,Pós-fixado referenciado em moeda estrangeira


In [18]:
df['TaxaJurosAoMes'] = df.apply(lambda x: trim_decimal(transform_taxa(x['TaxaJurosAoAno'], 1, 12)), axis=1)

In [21]:
SUBCLASSIFICACAO_REPLACE_MAP = {
	'PÓS-FIXADO': 'Pós-fixado',
	'PRÉ-FIXADO': 'Pré-fixado',
	'PÓS-FIXADO REFERENCIADO EM IPCA': 'Pós-fixado referenciado em IPCA',
	'PÓS-FIXADO REFERENCIADO EM TR': 'Pós-fixado referenciado em TR',
}
df['SubClassificacao'] = df['SubClassificacao'].replace(SUBCLASSIFICACAO_REPLACE_MAP)

In [22]:
df['SubClassificacao'].value_counts()

SubClassificacao
Pré-fixado                                      2016496
Pós-fixado referenciado em juros flutuantes      284465
Pós-fixado referenciado em moeda estrangeira     106677
Pós-fixado referenciado em TR                      3544
Pós-fixado referenciado em IPCA                     528
Name: count, dtype: int64

In [25]:
df['Modalidade'].value_counts()

Modalidade
Capital de giro com prazo até 365 dias            230590
Crédito pessoal não-consignado                    223087
Capital de giro com prazo superior a 365 dias     217137
Conta garantida                                   217096
Cheque especial                                   168685
Crédito pessoal consignado privado                155191
Desconto de duplicatas                            152863
Aquisição de veículos                             141057
Cartão de crédito - rotativo total                132411
Crédito pessoal consignado público                125878
Aquisição de outros bens                          124869
Cartão de crédito - parcelado                     120252
Crédito pessoal consignado INSS                   115134
Adiantamento sobre contratos de câmbio (ACC)      106677
Desconto de cheques                                95385
Vendor                                             40298
Antecipação de faturas de cartão de crédito        28354
Arrendamento mercant